In [36]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
print(selenium.__version__)

3.141.0


In [37]:
#Chrome 브라우저를 제어하는 데 필요한 옵션들을 설정
chrome_options = Options() 

#브라우저를 수동으로 제어하고자 할 때 유용
chrome_options.add_experimental_option("detach", True) 

#특정 스위치를 비활성화할 수 있습니다. 불필요한 로깅 메시지가 출력X
chrome_options.add_experimental_option("excludeSwitches",["enable-logging"]) 

#위에서 설정한 옵션들을 사용하여 Chrome 브라우저의 WebDriver 인스턴스를 생성
driver = webdriver.Chrome(options=chrome_options) 

In [38]:
# 무신사 코디맵 1페이지
url = 'https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1'
driver.get(url)
driver.maximize_window() # 브라우저의 크기 전체화면으로 확대

In [39]:
columns = ['codimap_category', 'codimap_title', 'codimap_date', 'views', 'comment_numbers']
values = []
soup = BeautifulSoup(driver.page_source, 'lxml')  
data_rows = soup.find_all('li', attrs={'class':'style-list-item'})

for i, row in enumerate(data_rows):
    
    blank = []
        
    category = row.find('span', attrs={'class':'style-list-information__text'}) # 코디 카테고리 가져오기
    if category:                                                                # 해당 조건에 맞는 정보가 하나라도 있을 경우:
        category = category.get_text().strip()                                  # html에서 텍스트만 가져온 뒤 공백 제거
        blank.append(category)
    else:                                                                       # 해당 조건에 맞는 정보가 하나도 없을 경우:
        blank.append('Something is wrong')
        print('{}번째 코디 카테고리 가져올때 문제발생'.format(i+1))
        continue
    
    
    title = row.find('strong', attrs={'class':'style-list-information__title'}) # 코디 이름 가져오기
    if title:
        title = title.get_text().strip()
        blank.append(title)
    else:
        blank.append('Something is wrong')
        print('{}번째 코디 이름 가져올때 문제발생'.format(i+1))
        continue
    
    
    date = row.find('span', attrs={'class':'post-information__text'})        # 날짜 가져오기
    if date:
        date = date.get_text().strip()    
        blank.append(date)
    else:
        blank.append('Something is wrong')
        print('{}번째 날짜 가져올때 문제발생'.format(i+1))
        continue
    
    # 추천수 및 댓글 
    post_info_div = soup.find_all('div', class_='post-information')
    first_div = post_info_div[i]
    spans = first_div.find_all('span', class_='post-information__text') # 해당 div 요소에서 span 태그를 찾아 텍스트 값을 리스트에 추가
    for j, span in enumerate(spans):
        if j != 0:
            # 숫자 부분만 추출
            space_split = span.get_text().split(' ')
            if len(space_split) > 1:
                blank.append(space_split[1])
        
    values.append(blank)

In [40]:
values

[['캐주얼', '톤온톤 코디', '21.07.02', '17,200', '50'],
 ['캐주얼', '블랙 매니아', '21.02.23', '36,256', '47'],
 ['캐주얼', '오늘의 코디', '22.03.22', '13,348', '43'],
 ['댄디', '추천 댄디 룩', '21.09.15', '21,545', '32'],
 ['캐주얼', '릴랙스 룩', '21.02.23', '23,993', '25'],
 ['스트릿', '힙하다 힙해', '21.02.03', '36,031', '24'],
 ['댄디', '데이트 코디', '21.12.24', '10,931', '19'],
 ['캐주얼', '심플한 코디', '21.07.30', '41,320', '18'],
 ['댄디', '훈훈 남친 룩', '21.03.24', '19,969', '17'],
 ['댄디', '남친 룩으로 제격', '21.12.24', '10,155', '17'],
 ['댄디', '서머 데이트 룩', '21.07.26', '8,342', '17'],
 ['캐주얼', '지금 입기 좋아', '21.08.31', '20,863', '16'],
 ['캐주얼', '꾸안꾸 코디', '22.11.04', '5,171', '15'],
 ['아메카지', '아메리칸 캐주얼', '21.12.21', '5,813', '14'],
 ['비즈니스캐주얼', '깔끔하게 즐겨', '21.11.09', '6,581', '13'],
 ['아메카지', '남다른 감각', '21.11.17', '5,057', '11'],
 ['캐주얼', '편안함의 완성', '21.10.04', '20,054', '11'],
 ['비즈니스캐주얼', '매력적인 포멀 룩', '21.09.17', '11,702', '10'],
 ['캐주얼', '원 마일 웨어', '21.10.08', '17,049', '10'],
 ['캐주얼', '편하게 완성', '21.07.21', '15,319', '10'],
 ['캐주얼', '레이어드의 매력', '21

In [41]:
df = pd.DataFrame(values, columns = columns)

In [42]:
df

,codimap_category,codimap_title,codimap_date,views,comment_numbers
0,캐주얼,톤온톤 코디,21.07.02,"17,200",50
1,캐주얼,블랙 매니아,21.02.23,"36,256",47
2,캐주얼,오늘의 코디,22.03.22,"13,348",43
3,댄디,추천 댄디 룩,21.09.15,"21,545",32
4,캐주얼,릴랙스 룩,21.02.23,"23,993",25
5,스트릿,힙하다 힙해,21.02.03,"36,031",24
6,댄디,데이트 코디,21.12.24,"10,931",19
7,캐주얼,심플한 코디,21.07.30,"41,320",18
8,댄디,훈훈 남친 룩,21.03.24,"19,969",17
9,댄디,남친 룩으로 제격,21.12.24,"10,155",17


In [43]:
df.to_csv('output.csv', index=False, encoding='cp949')